# Inputs
---

In [ ]:
import numpy as np, zarr, nrrd

p = '/nrs/scicompsoft/rokicki/multifish/UCLA-Zipursky'
fix_path = p + '/lt185_r1/export.n5'
mov_path = p + '/lt185_r2/export.n5'
exp_factor = 1  # replace this with the known expansion factor for your sample to use pre-expansion units

# load fix data and spacing
fix_zarr = zarr.open(store=zarr.N5Store(fix_path), mode='r')
fix_meta = fix_zarr.attrs.asdict()
fix_spacing = np.array(fix_meta['pixelResolution']['dimensions'][::-1]) / exp_factor
fix_spacing_s1 = fix_spacing * [1, 2, 2]
fix_spacing_s3 = fix_spacing * [4, 8, 8]
fix_spacing_s4 = fix_spacing * [8, 16, 16]

# load mov data and spacing
mov_zarr = zarr.open(store=zarr.N5Store(mov_path), mode='r')
mov_meta = mov_zarr.attrs.asdict()
mov_spacing = np.array(mov_meta['pixelResolution']['dimensions'][::-1]) / exp_factor
mov_spacing_s1 = mov_spacing * [1, 2, 2]
mov_spacing_s3 = mov_spacing * [4, 8, 8]
mov_spacing_s4 = mov_spacing * [8, 16, 16]

# print spacings
print(fix_spacing, mov_spacing)
print(fix_spacing_s1, mov_spacing_s1)
print(fix_spacing_s3, mov_spacing_s3)
print(fix_spacing_s4, mov_spacing_s4)

# print shapes
print(fix_zarr['/c3/s0'].shape, mov_zarr['/c3/s0'].shape)

# # write some channels
# nrrd.write('./fix.nrrd', fix_zarr['/c3/s4'][...].transpose(2,1,0), compression_level=2)
# nrrd.write('./mov.nrrd', mov_zarr['/c3/s4'][...].transpose(2,1,0), compression_level=2)

# Alignment
---

### global

In [ ]:
# alignment functions
from bigstream.align import alignment_pipeline
from bigstream.transform import apply_transform

# get global alignment channels
fix = fix_zarr['/c3/s4'][...]
mov = mov_zarr['/c3/s4'][...][:, ::-1, ::-1]  # moving image data is 180 degrees rotates about z axis

# create some masks to focus alignment only on planes below the large artifact at the top
fix_mask = np.ones(fix.shape, dtype=np.uint8)
fix_mask[:65] = 0
mov_mask = np.ones(mov.shape, dtype=np.uint8)
mov_mask[:45] = 0

# define alignment steps
ransac_kwargs = {
    'blob_sizes':[2, 8],
    'cc_radius':12,
    'match_threshold':0.4,
    'nspots':10000,
}

affine_kwargs = {
    'alignment_spacing':4.0,
    'shrink_factors':(1,),
    'smooth_sigmas':(4.,),
    'optimizer_args':{
        'learningRate':0.25,
        'minStep':0.,
        'numberOfIterations':400,
    },
    'metric':'C',
}

steps = [('ransac', ransac_kwargs,),
         ('affine', affine_kwargs,),]

# align
affine = alignment_pipeline(
    fix, mov,
    fix_spacing_s4,
    mov_spacing_s4,
    steps,
    fix_mask=fix_mask,
    mov_mask=mov_mask,
)

# apply affine only
affine_aligned = apply_transform(
    fix, mov,
    fix_spacing_s4, mov_spacing_s4,
    transform_list=[affine,],
)

# write results
np.savetxt('affine.mat', affine)
nrrd.write('./affine.nrrd', affine_aligned.transpose(2,1,0), compression_level=2)

# load precomputed results
affine = np.loadtxt('./affine.mat')

### local

In [ ]:
from bigstream.piecewise_align import distributed_piecewise_alignment_pipeline
from bigstream.transform import apply_transform

# get global alignment channels
fix = fix_zarr['/c3/s3'][...]
mov = mov_zarr['/c3/s3'][...][:, ::-1, ::-1]  # moving image data is 180 degrees rotates about z axis

# define alignment steps
ransac_kwargs = {
    'blob_sizes':[4, 16],
    'cc_radius':16,
    'match_threshold':0.4,
    'nspots':4000,
}

affine_kwargs = {
    'alignment_spacing':2.0,
    'shrink_factors':(2,),
    'smooth_sigmas':(4.,),
    'optimizer_args':{
        'learningRate':0.25,
        'minStep':0.,
        'numberOfIterations':400,
    },
    'metric':'C',
}

steps = [('ransac', ransac_kwargs,),
         ('affine', affine_kwargs,),]

blocksize = [256, 256, 256]

cluster_kwargs = {
    'project':'scicompsoft',
    'ncpus':4,
    'threads':1,
    'min_workers':60,
    'max_workers':60,
}

# align
deform = distributed_piecewise_alignment_pipeline(
    fix, mov,
    fix_spacing_s3,
    mov_spacing_s3,
    steps,
    blocksize,
    static_transform_list=[affine,],
    cluster_kwargs=cluster_kwargs,
)

# apply affine only
deform_aligned = apply_transform(
    fix, mov,
    fix_spacing_s3, mov_spacing_s3,
    transform_list=[affine, deform],
)

# write results
nrrd.write('./deform.nrrd', deform, compression_level=2)
nrrd.write('./deformed.nrrd', deform_aligned.transpose(2,1,0), compression_level=2)

# load precomputed results
deform, _ = nrrd.read('./deform.nrrd')

### even more local

In [ ]:
# get global alignment channels
fix = fix_zarr['/c3/s3'][...]
mov = mov_zarr['/c3/s3'][...][:, ::-1, ::-1]  # moving image data is 180 degrees rotates about z axis

deform_kwargs = {
    'alignment_spacing':2.0,
    'shrink_factors':(2,),
    'smooth_sigmas':(4.,),
    'control_point_spacing':200.0,
    'control_point_levels':(1,),
    'optimizer_args':{
        'learningRate':2.5,
        'minStep':0.07,
        'numberOfIterations':75,
    },
    # 'metric':'C',  # correlation required too much memory, but the default MI worked well
}

steps = [('deform', deform_kwargs,),]
         
blocksize = [256, 256, 256]

cluster_kwargs = {
    'project':'scicompsoft',
    'ncpus':3,
    'threads':1,
    'min_workers':60,
    'max_workers':60,
    'config':{
        'distributed.worker.memory.target':0.9,
        'distributed.worker.memory.spill':0.9,
        'distributed.worker.memory.pause':0.9,
    },
}

# align
deform2 = distributed_piecewise_alignment_pipeline(
    fix, mov,
    fix_spacing_s3,
    mov_spacing_s3,
    steps,
    blocksize,
    static_transform_list=[affine, deform],
    cluster_kwargs=cluster_kwargs,
)

# apply affine only
deform2_aligned = apply_transform(
    fix, mov,
    fix_spacing_s3, mov_spacing_s3,
    transform_list=[affine, deform, deform2],
)
         
# write results
nrrd.write('./deform2.nrrd', deform2, compression_level=2)
nrrd.write('./deformed2.nrrd', deform2_aligned.transpose(2,1,0), compression_level=2)

# load precomputed results
deform2, _ = nrrd.read('./deform2.nrrd')

## Histogram equalize data to visualize alignment result better
---

In [ ]:
from skimage.exposure import equalize_adapthist, equalize_hist
from scipy.ndimage import gaussian_filter, grey_opening

fix_s3, _ = nrrd.read('./fix_s3.nrrd')
deformed, _ = nrrd.read('./deformed.nrrd')
deformed2, _ = nrrd.read('./deformed2.nrrd')

fix_s3_corrected = equalize_hist(fix_s3, mask=(fix_s3 > 90))
fix_s3_corrected = np.round(fix_s3_corrected * 64000).astype(np.uint16)

deformed_corrected = equalize_hist(deformed, mask=(deformed > 90))
deformed_corrected = np.round(deformed_corrected * 64000).astype(np.uint16)

deformed2_corrected = equalize_hist(deformed2, mask=(deformed2 > 90))
deformed2_corrected = np.round(deformed2_corrected * 64000).astype(np.uint16)

nrrd.write('./fix_s3_corrected.nrrd', fix_s3_corrected, compression_level=2)
nrrd.write('./deformed_corrected.nrrd', deformed_corrected, compression_level=2)
nrrd.write('./deformed2_corrected.nrrd', deformed2_corrected, compression_level=2)